In [1]:
%reload_ext autoreload
%autoreload 2

import os
import pprint

import pymongo

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource
from bokeh.palettes import all_palettes

from climatools.atm import *
from climatools.dataset import *
from climatools.spectrum import *
from climatools.html.html import *
from climatools.plot.plot import *
from climatools.lblnew.bestfit_params import *

from IPython import display

In [2]:
output_notebook()
client = pymongo.MongoClient('localhost', 27017)

Loading BokehJS ...

## Data

In [3]:
MOL = 'co2'
COMMITNUMBER = '1013f91'
ATMPRO = dict(mls=294, saw=257, trp=300)

In [4]:
atmcomp = AtmComposition.cliradlw_nongreys(onlygas=MOL)

In [5]:
DCLI = {}
for atmpro, _ in ATMPRO.items():
    pcli = atmcomp.to_cliradparam(commitnumber=COMMITNUMBER, atmpro=atmpro, tsfc=ATMPRO[atmpro])
    dcli = pcli.modeldata_pymongo(collection=client.cliradnew.lw)
    DCLI[atmpro] = dcli

In [6]:
DCRD = {}

In [16]:
atmpro = 'trp'
pcrds = atmcomp.to_lblnewparam(bestfit_values=True, atmpro=atmpro, tsfc=ATMPRO[atmpro])

In [17]:
cmts = [p.avail_commits(client.lblnew.bestfit_lw) for p in pcrds]
cmts

[['a22ab94'], ['a22ab94'], ['a22ab94'], ['a22ab94'], ['a22ab94'], ['a22ab94']]

In [18]:
commits = ['a22ab94', 'a06b618', '5014a19']
dcrds = [p.modeldata_pymongo(collection=client.lblnew.bestfit_lw, commits=commits) for p in pcrds]

dcrd = concat(*dcrds)
DCRD[atmpro] = dcrd

In [19]:
DCLI, DCRD

({'mls': <climatools.dataset.CliradnewLWModelData at 0x2b3a67164a58>,
  'saw': <climatools.dataset.CliradnewLWModelData at 0x2b3aac5672e8>,
  'trp': <climatools.dataset.CliradnewLWModelData at 0x2b3aac56ca58>},
 {'mls': <climatools.dataset.LBLnewModelData at 0x2b3aac81d470>,
  'saw': <climatools.dataset.LBLnewModelData at 0x2b3a802f17b8>,
  'trp': <climatools.dataset.LBLnewModelData at 0x2b3aadaba8d0>})

## Analysis code

In [20]:
def show_html(s):
    display.display(display.HTML(s))

def show_markdown(s):
    display.display(display.Markdown(s))

In [21]:
def show_parameters(dcrd, dcli):
    srss = [pd.Series(dcli.param, name=':D')]
    srss_crd = [pd.Series(p) for p in dcrd.param]
    df = pd.concat(srss_crd, axis=1, sort=True, keys=[f"band{s['band']} {s['molecule']}" for s in srss_crd])
    srss.append(df)
    df = pd.concat(srss, axis=1, sort=True, keys=['clirad-lw', 'lblnew-bestfit lw'])
    df.fillna('-', inplace=True)
    with pd.option_context('display.max_colwidth', 100):
        display.display(df)

In [22]:
import rtmtools.clirad.sw.wrangle as cliradwrangle

def oldclirad_data_atm():
    '''
    Load the OLD clirad's results. mls only.
    '''
    fpath = os.path.join('/chia_cluster/home/jackyu/radiation',
                         'clirad-lw',
                         'LW',
                         'examples',
                         'mls75_h2o_atmpro_co2_.0004_o3_atmpro_n2o_3.2e-7_ch4_1.8e-6_H2012',
                         'OUTPUT_CLIRAD.dat')
    
    ds = cliradwrangle.load_OUTPUT_CLIRAD(readfrom=fpath)
    
    ds_cool = xr.Dataset()
    ds_cool.coords['pressure'] = ('pressure', ds['layer_pressure'])
    ds_cool.coords['band'] = ('band', ds['spectral_band'])
    ds_cool['coolrg'] = (('band', 'pressure'), - ds['heating_rate'])
    
    ds_flux = xr.Dataset()
    ds_flux.coords['pressure'] = ('pressure', ds['level_pressure'])
    ds_flux.coords['band'] = ('band', ds['spectral_band'])
    ds_flux['flug'] = (('band', 'pressure'), ds['flux_up'])
    ds_flux['fldg'] = (('band', 'pressure'), ds['flux_down'])
    ds_flux['fnetg'] = (('band', 'pressure'), ds['net_flux'])
    
    
    d = {}
    d['cool'] = ds_cool
    d['flux'] = ds_flux
    return d

In [23]:
def show_makeup():
    df = pd.DataFrame()
    for band, gs in atmcomp.gasinbands.items():
        for g in gs:
            df.loc[g, band] = str(atmcomp.gasconcs[g])

    df = df.fillna(0)
    df.columns.name = 'clirad band'
    df.index.name = 'molecule'
    
    display.display(df)
    display.display(display.Markdown('*TABLE.* Non-grey absorbers in the atmosphere.'))

In [24]:
def pltdata_cool(dcrd=None, dcli=None, atmpro='mls'):
    colors = all_palettes['Set1'][4]
    cool_cli = dcli.wgt_cool.sel(i=1).sum('band')['coolrg']
    cool_crd = dcrd.crd_cool['coolrg'].sum('band').sum('g')
    
    data = [
        {'label': 'CLIRAD',
         'srs': cool_cli,
         'line_dash': 'dashed', 'line_width': 5,
         'color': colors[0], 'alpha': .6},
        {'label': 'CRD',
         'srs': cool_crd,
         'line_dash': 'solid', 'line_width': 1.5,
         'marker': 'circle', 'marker_size': 5,
         'color': colors[2], 'alpha': 1}
    ]
    
    # include old CLIRAD's results for mls profile
#    if atmpro == 'mls':
#        d_oldclirad = oldclirad_data_atm()
#        ds_oldclirad = d_oldclirad['cool']        
#        data.append(
#            {'label': 'old CLIRAD (H2012)',
#             'srs': fmt_cool(ds_oldclirad),
#             'line_dash': 'solid', 'line_width': 1.5,
#             'marker': 'square', 'marker_size': 3,
#             'color': colors[3], 'alpha': .5})
    return data


def pltdata_cooldiff(dcrd=None, dcli=None, atmpro='mls'): 
    colors = all_palettes['Set1'][4]
    cool_cli = dcli.wgt_cool.sel(i=1).sum('band')['coolrg']
    cool_crd = dcrd.crd_cool['coolrg'].sum('band').sum('g')
    cool_dif = cool_cli - cool_crd
    
    data = [
        {'label': 'CLIRAD - CRD',
         'srs': cool_dif,
         'line_dash': 'solid', 'line_width': 1.5, 
         'marker': 'circle', 'marker_size': 7,
         'color': colors[3], 'alpha': .8}
    ]
    
    # include old CLIRAD's results for mls profile
#    if atmpro == 'mls':
#        d_oldclirad = oldclirad_data_atm()
#        ds_oldclirad = d_oldclirad['cool']
#        ds_oldclirad.coords['pressure'] = ds_crd.coords['pressure']
#        ds_diff_old = ds_oldclirad.sum('band') - ds_crd.sum('band')
#        data.append(
#            {'label': 'old CLIRAD (H2012) - CRD',
#             'srs': fmt_cool(ds_diff_old),
#             'line_dash': 'dashed', 'line_width': 4,
#             'color': colors[1], 'alpha': .5}
#        )
    return data


def show_cool(dcrd=None, dcli=None, atmpro='mls'):
    data_cool = pltdata_cool(dcrd, dcli, atmpro=atmpro)
    p_cool_liny = plt_vert_profile_bokeh(pltdata=data_cool)
    p_cool_logy = plt_vert_profile_bokeh(pltdata=data_cool, y_axis_type='log', prange=(.01, 200))
    
    data_cooldiff = pltdata_cooldiff(dcrd, dcli, atmpro=atmpro)
    p_cooldiff_logy = plt_vert_profile_bokeh(pltdata=data_cooldiff, y_axis_type='log', prange=(.01, 200))
    
    everything = gridplot(p_cool_liny, p_cool_logy, p_cooldiff_logy, ncols=3)
    show(everything)
    display.display(display.Markdown('*FIGURE.* Cooling rates & difference.'))

In [25]:
def hist_band_vs_flux(da, title='Title'):
    bands = [str(b.values) for b in da['band']]
    source = ColumnDataSource(data={'band': bands, 'flux': da.values})
    p = figure(x_range=bands, title=title)
    p.vbar(source=source, x='band', top='flux', width=.9)
    p.yaxis.axis_label = 'flux (W m-2)'
    p.xaxis.axis_label = 'spectral band'
    return p

def show_hist_flux(dcrd=None, dcli=None, atmpro='mls'):
    ds_crd = dcrd.crd_flux.sum('g')
    ds_clirad = dcli.wgt_flux.sel(i=1)
    
    ip, varname = 0, 'flug'
    da = (ds_clirad - ds_crd).isel(pressure=ip)[varname]
    p_toa = hist_band_vs_flux(da, title='TOA flux. CLIRAD - CRD.')

    ip, varname = -1, 'fldg'
    da = (ds_clirad - ds_crd).isel(pressure=ip)[varname]
    p_sfc = hist_band_vs_flux(da, title='SFC flux. CLIRAD - CRD.')    
    
    atm_crd = (ds_crd.isel(pressure=0) - ds_crd.isel(pressure=-1))['fnetg']
    atm_clirad = (ds_clirad.isel(pressure=0) - ds_clirad.isel(pressure=-1))['fnetg']
    da = atm_clirad - atm_crd
    p_atm = hist_band_vs_flux(da, title='Atmosphere heating. CLIRAD - CRD.')

    everything = gridplot(p_toa, p_sfc, p_atm, ncols=3, plot_width=300, plot_height=300)
    
    show(everything)
    display.display(
        display.Markdown('*FIGURE.* Difference between CLIRAD and CRD'
          ' in TOA, SFC and net atmosphere flux,'
          ' in each spectral band.'))

def tb_flux(fluxcrd, fluxcli, atmpro='mls'):
    '''
    Return table that shows the difference between CRD and CLIRAD fluxes
    at various levels.
    '''
    tropos = tropopause_pressures()
    
    olr_crd = fluxcrd['flug'].isel(pressure=0)
    sfc_crd = fluxcrd['fldg'].isel(pressure=-1)
    atm_crd = (fluxcrd.isel(pressure=0) - fluxcrd.isel(pressure=-1))['fnetg']
    trop_up_crd = fluxcrd['flug'].sel(pressure=tropos[atmpro])
    trop_down_crd = fluxcrd['fldg'].sel(pressure=tropos[atmpro])
    trop_net_crd = fluxcrd['fnetg'].sel(pressure=tropos[atmpro])
    
    olr_cli = fluxcli['flug'].isel(pressure=0)
    sfc_cli = fluxcli['fldg'].isel(pressure=-1)
    atm_cli = (fluxcli.isel(pressure=0) - fluxcli.isel(pressure=-1))['fnetg']
    trop_up_cli = fluxcli['flug'].sel(pressure=tropos[atmpro])
    trop_down_cli = fluxcli['fldg'].sel(pressure=tropos[atmpro])
    trop_net_cli = fluxcli['fnetg'].sel(pressure=tropos[atmpro])
    
    df = pd.DataFrame()
    df.loc['CLIRAD - CRD', 'TOA up'] = (olr_cli - olr_crd).values
    df.loc['CLIRAD - CRD', 'SFC down'] = (sfc_cli - sfc_crd).values
    df.loc['CLIRAD - CRD', 'ATM heating'] = (atm_cli - atm_crd).values
    df.loc['CLIRAD - CRD', 'tropo up'] = (trop_up_cli - trop_up_crd).values
    df.loc['CLIRAD - CRD', 'tropo down'] = (trop_down_cli - trop_down_crd).values
    df.loc['CLIRAD - CRD', 'tropo net'] = (trop_net_cli - trop_net_crd).values
    df.loc['CRD', 'TOA up'] = olr_crd.values
    df.loc['CRD', 'SFC down'] = sfc_crd.values
    df.loc['CRD', 'ATM heating'] = atm_crd.values
    df.loc['CRD', 'tropo up'] = trop_up_crd.values
    df.loc['CRD', 'tropo down'] = trop_down_crd.values
    df.loc['CRD', 'tropo net'] = trop_net_crd.values
        
    return df.astype('float').round(2)   
    
def show_tb_flux(dcrd=None, dcli=None, atmpro='mls'):
    dfs = {}
    fluxcrd = dcrd.crd_flux.sum(['band', 'g'])    
    fluxcli = dcli.wgt_flux.sel(i=1).sum('band')
    dfs['Sum over band'] = tb_flux(fluxcrd, fluxcli, atmpro=atmpro)
    for b in dcrd.crd_flux['band']:
        fluxcrd = dcrd.crd_flux.sum('g').sel(band=b)
        fluxcli = dcli.wgt_flux.sel(i=1, band=b)
        dfs[f'Band {int(b)}'] = tb_flux(fluxcrd, fluxcli, atmpro=atmpro)
    dflist = [df for _, df in dfs.items()]
    keys = [key for key, _ in dfs.items()]
    df = pd.concat(dflist, axis=0, keys=keys)
    
#    if atmpro == 'mls':
#        ds_oldclirad = oldclirad_data_atm()['flux']
#        ds_oldclirad['pressure'] = ds_crd['pressure']
#        olr_oldclirad = ds_oldclirad['flug'].isel(pressure=0)
#        sfc_oldclirad = ds_oldclirad['fldg'].isel(pressure=-1)
#        atm_oldclirad = (ds_oldclirad.isel(pressure=0)
#                         - ds_oldclirad.isel(pressure=-1))['fnetg']
   
##    if atmpro == 'mls':
##        df.loc['old CLIRAD - CRD', 'OLR flux'] = (fmt(olr_oldclirad) - fmt(olr_crd)).values
##        df.loc['old CLIRAD - CRD', 'SFC flux'] = (fmt(sfc_oldclirad) - fmt(sfc_crd)).values
##        df.loc['old CLIRAD - CRD', 'ATM heating'] = (fmt(atm_oldclirad) - fmt(atm_crd)).values
    display.display(df)
    display.display(
        display.Markdown('*TABLE.* Difference between CLIRAD and CRD fluxes'
          ' at various atmosphere levels.'))
    
#show_hist_flux(dcrd, dcli, atmpro='mls')
#show_tb_flux(DCRD['mls'], DCLI['mls'], atmpro='mls')

In [26]:
def cool_tofile(atmpro=None):
    dcli = clirad_data_atm(params_atm=[subparam_atm_clirad(atmpro=atmpro)])
    dcrd = crd_data_atm(params_atm=subparams_atm_lblnew(atmpro=atmpro))
    
    dcli = dcli['cool'].sel(i=1).drop('i').sum('band')
    dcrd = dcrd['cool'].sum('band')
    
    dcli = dcli['coolrg'].to_dataframe()
    dcrd = dcrd['coolrg'].to_dataframe()
    
    dcli = dcli.set_index('layer', append=True)
    dcrd = dcrd.set_index('layer', append=True)
    
    ddif = dcli - dcrd
    
    df = pd.concat([dcrd, dcli, ddif], axis=1, keys=['CRD', 'CLIRAD', 'CLIRAD - CRD'])
    return df


def flux_tofile(atmpro=None):
    tropopauses = {'mls': 40, 'saw': 45, 'trp': 37}
    ilevels = [0, tropopauses[atmpro], -1]
    
    dcli = clirad_data_atm(params_atm=[subparam_atm_clirad(atmpro=atmpro)])
    dcrd = crd_data_atm(params_atm=subparams_atm_lblnew(atmpro=atmpro))
    
    dcli = dcli['flux'].sel(i=1).drop('i').sum('band').isel(pressure=ilevels)
    dcrd = dcrd['flux'].sum('band').isel(pressure=ilevels)
    
    dcli = dcli.to_dataframe()
    dcrd = dcrd.to_dataframe()
    
    dcli = dcli.set_index('level', append=True)
    dcrd = dcrd.set_index('level', append=True)
    
    ddif = dcli - dcrd
    
    df = pd.concat([dcrd, dcli, ddif], axis=1, keys=['CRD', 'CLIRAD', 'CLIRAD - CRD'])
    return df


def script_tofile():
    fname = f'results_{MOL}atm_cliradsinglerun.xlsx'
    writer = pd.ExcelWriter(fname)
    for atmpro in ('mls', 'saw', 'trp'):
        dflux = flux_tofile(atmpro=atmpro)
        dflux.to_excel(writer, f'flux {atmpro}')
        
        dcool = cool_tofile(atmpro=atmpro)
        dcool.to_excel(writer, f'cool {atmpro}')
        
    html = f'Download: <a href="./{fname}">{fname}</a>'
    show_html(html) 

In [27]:
def script():
    
    d_atm = {'mls': 'mid-latitude summer', 'saw': 'sub-arctic winter', 'trp': 'tropical'}
    
    s_makeup = 'Makeup of atmosphere.'
    s_parameters = 'Parameters of runs.'
    s_atmpro = '# {}'
    s_cool = 'Cooling rates. {}.'
    s_flux = 'Fluxes. {}.'
    
    atmpros = ['mls', 'saw', 'trp']
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(getHTML_hrefanchor(s_makeup))
    
    for atmpro in atmpros:
        show_markdown('**' + d_atm[atmpro] + '**')
        show_html(getHTML_hrefanchor(s_parameters + f' {atmpro}'))
        show_html(getHTML_hrefanchor(s_cool.format(atmpro)))
        show_html(getHTML_hrefanchor(s_flux.format(atmpro)))

    # Atmosphere makeup
    show_html(getHTML_idanchor(s_makeup))
    show_markdown(getMarkdown_sectitle(s_makeup))
    show_makeup()
        
    for atmpro in atmpros:
        dcrd, dcli = DCRD[atmpro], DCLI[atmpro]
        
        show_html(getHTML_idanchor(s_parameters + f' {atmpro}'))
        show_markdown(getMarkdown_sectitle(s_parameters + f' {atmpro}'))
        show_parameters(dcrd, dcli)
        
        show_html(getHTML_idanchor(s_cool.format(atmpro)))
        show_markdown(getMarkdown_sectitle(s_cool.format(atmpro)))
        show_cool(dcrd, dcli, atmpro=atmpro)
        
        show_html(getHTML_idanchor(s_flux.format(atmpro)))
        show_markdown(getMarkdown_sectitle(s_flux.format(atmpro)))
        show_hist_flux(dcrd, dcli, atmpro=atmpro)
        show_tb_flux(dcrd, dcli, atmpro=atmpro)
    
    #script_tofile()

## Analysis Result

In [28]:
script()

### Table of Contents

**mid-latitude summer**

**sub-arctic winter**

**tropical**

# Makeup of atmosphere.

...

*FIGURE.* Cooling rates & difference.

# Fluxes. trp.

*FIGURE.* Difference between CLIRAD and CRD in TOA, SFC and net atmosphere flux, in each spectral band.

TOA up  SFC down  ATM heating  tropo up  \
Sum over band CLIRAD - CRD    0.68      0.67         0.01      0.76   
              CRD          -182.34     85.22       -36.52   -179.02   
Band 3        CLIRAD - CRD   -0.12      0.04        -0.16     -0.08   
              CRD           -30.87     17.95       -10.33    -30.90   
Band 4        CLIRAD - CRD    0.42      0.10         0.32      0.43   
              CRD           -16.25     46.47       -15.66    -12.86   
Band 5        CLIRAD - CRD    0.37      0.49        -0.12      0.39   
              CRD           -27.53     16.47        -8.78    -27.56   
Band 6        CLIRAD - CRD   -0.04     -0.07         0.02     -0.05   
              CRD           -66.83      1.43        -0.92    -66.85   
Band 7        CLIRAD - CRD   -0.04     -0.05         0.02     -0.04   
              CRD           -34.14      1.41        -0.82    -34.16   
Band 11       CLIRAD - CRD    0.10      0.17        -0.07      0.11   
              CRD            -6.71      1.49        -0.01     -6.69   

                            tropo down  tropo net  
Sum over band CLIRAD - CRD        0.02       0.78  
              CRD                 7.24    -171.78  
Band 3        CLIRAD - CRD        0.02      -0.06  
              CRD                 0.65     -30.25  
Band 4        CLIRAD - CRD        0.03       0.46  
              CRD                 6.15      -6.71  
Band 5        CLIRAD - CRD       -0.03       0.35  
              CRD                 0.41     -27.15  
Band 6        CLIRAD - CRD       -0.00      -0.05  
              CRD                 0.01     -66.84  
Band 7        CLIRAD - CRD       -0.00      -0.04  
              CRD                 0.01     -34.15  
Band 11       CLIRAD - CRD       -0.00       0.11  
              CRD                 0.00      -6.68

*TABLE.* Difference between CLIRAD and CRD fluxes at various atmosphere levels.

## --

In [29]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')